In [1]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchsummary import summary
from torchvision.transforms import ToTensor,v2,Grayscale
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_1 = nn.Sequential(
          nn.Conv2d(1, 16, 3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2))
        self.conv_layer_2 = nn.Sequential(
          nn.Conv2d(16, 32, 3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout(0.2))
        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=32*30*30, out_features=128),
          nn.ReLU(),
          nn.Dropout(0.05),
          nn.Linear(in_features=128, out_features=6))
    def forward(self, x: torch.Tensor):
        x = self.conv_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.classifier(x)
        return x

In [3]:
model = ImageClassifier()
model.to('cuda')
model.load_state_dict(torch.load("./weight/black095.pth"))
summary(model ,(1,128,128))
model.eval()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             160
              ReLU-2         [-1, 16, 126, 126]               0
         MaxPool2d-3           [-1, 16, 63, 63]               0
            Conv2d-4           [-1, 32, 61, 61]           4,640
              ReLU-5           [-1, 32, 61, 61]               0
         MaxPool2d-6           [-1, 32, 30, 30]               0
           Dropout-7           [-1, 32, 30, 30]               0
           Flatten-8                [-1, 28800]               0
            Linear-9                  [-1, 128]       3,686,528
             ReLU-10                  [-1, 128]               0
          Dropout-11                  [-1, 128]               0
           Linear-12                    [-1, 6]             774
Total params: 3,692,102
Trainable params: 3,692,102
Non-trainable params: 0
---------------------------

ImageClassifier(
  (conv_layer_1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer_2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=28800, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.05, inplace=False)
    (4): Linear(in_features=128, out_features=6, bias=True)
  )
)

In [8]:
def totensor(result_image):
    to_tensor = ToTensor()
    tensor_image = to_tensor(result_image)
    tensor_image = tensor_image.unsqueeze(0)
    #tensor_image.shape
    return tensor_image

In [5]:
def predict(tensor_image):
    out = model(tensor_image.to('cuda'))
    soft=nn.Softmax(dim=1)
    out = soft(out)
    print(out)
    max_value, max_index = torch.max(out, dim=1)

    print("predict number", max_index.item())
    if max_index.item() == 1:
        cv2.imshow("predict",figure1)
    elif max_index.item() == 2:
        cv2.imshow("predict",figure2)
    elif max_index.item() == 3:
        cv2.imshow("predict",figure3)
    elif max_index.item() == 4:
        cv2.imshow("predict",figure4)
    elif max_index.item() == 5:
        cv2.imshow("predict",figure5)
    elif max_index.item() == 0:
        cv2.imshow("predict",figure0)

In [6]:
figure0 =cv2.imread('./RightLeft/0/zero1.jpg')
figure1 =cv2.imread('./RightLeft/1/one1.jpg')
figure2 =cv2.imread('./RightLeft/2/two1.jpg')
figure3 =cv2.imread('./RightLeft/3/three1.jpg')
figure4 =cv2.imread('./RightLeft/4/four1.jpg')
figure5 =cv2.imread('./RightLeft/5/five1.jpg')
cap = cv2.VideoCapture(0)
i=1
while 1:
    _ , FrameImage = cap.read()
    FrameImage = cv2.flip(FrameImage, 1)
    cv2.imshow("Webcam", FrameImage)
    
    hsi= cv2.cvtColor(FrameImage, cv2.COLOR_RGB2HSV)  #將圖片轉成HSI格式
    h, s, i = cv2.split(hsi)        #將圖片拆成分別為HSI的單通道圖片
    
    saturation_threshold = 70  # 調整需要的閾值

    # Saturation Slicing
    saturation_mask = s < saturation_threshold
    saturation_mask = saturation_mask.astype(np.uint8)

    saturation_mask = 1- saturation_mask 
    saturation_mask = saturation_mask * 255  
    result_image = cv2.resize(saturation_mask,(128,128),interpolation=cv2.INTER_AREA)
    tensor_image = totensor(result_image)
    cv2.imshow("mask",saturation_mask)
    print(tensor_image.shape)
    break
    predict(tensor_image)
    
    
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == ord('q'):
        break
    elif interrupt & 0xFF == ord('s'):
        path = './data/0/zero('+str(i)+').jpg'
        cv2.imwrite(path,FrameImage)
        i+=1

cap.release()
cv2.destroyAllWindows()

torch.Size([1, 1, 128, 128])


In [9]:
figure0 =cv2.imread('./BLACK/0/zero1.jpg')
figure1 =cv2.imread('./BLACK/1/one1.jpg')
figure2 =cv2.imread('./BLACK/2/two1.jpg')
figure3 =cv2.imread('./BLACK/3/three1.jpg')
figure4 =cv2.imread('./BLACK/4/four1.jpg')
figure5 =cv2.imread('./BLACK/5/five1.jpg')
from PIL import Image

img= cv2.cvtColor(figure0, cv2.COLOR_BGR2GRAY)  #將圖片轉成HSI格式
result_image = cv2.resize(img,(128,128),interpolation=cv2.INTER_AREA)

to_tensor = ToTensor()
tensor_image = to_tensor(result_image)
tensor_image = tensor_image.unsqueeze(1)
print(tensor_image.shape)
predict(tensor_image)

torch.Size([1, 1, 128, 128])
tensor([[1.0000e+00, 1.6077e-07, 1.1310e-08, 5.3467e-10, 6.1901e-10, 1.0443e-06]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
predict number 0
